### 차원 축소
- 차원의 저주 : 차원이 너무 많을 경우 훈련이 느려질 뿐만 아니라, 좋은 솔루션을 찾기 어렵게 만듦
- 또한, 차원을 축소한다면 데이터 시각화에도 많은 이점이 있음
- 두 가지 접근법 : 투영(projection)과 매니폴드학습(manifold learning)
- 훈련세트의 차원이 커질수록 각 샘플들의 거리가 멀어져, 과대적합 가능성이 커짐

##### 투영
- 대부분의 실전 문제는 모든 훈련 샘플이 고차원 공간 안의 저차원 부분공간에 놓여있음
- 3차원 샘플을 2차원 부분공간에 투영시키는 방법으로 차원 축소를 진행할 수 있음
- 하지만, 스위스 롤처럼 부분공간이 뒤틀리거나 휘어있다면 차원 축소시 스위스 롤의 층이 서로 뭉개지는 결과가 초래됨

##### 매니폴드
- 스위스 롤은 2D 매니폴드의 한 예시임 즉, 2D 매니폴드는 고차원 공간에서 휘어지거나 뒤틀린 2D모양
- 훈련 샘플이 놓여있는 매니폴드를 모델링하는 식으로 작동하는 것을 매니폴드 학습이라고 함
- 이는 대부분 실제 고차원 데이터셋이 더 낮은 저차원 매니폴드에 가깝게 놓여있다는 매니폴드 가정에 근거함
- 대부분 맞지만, 이런 암묵적 가정이 항상 유효한 것은 아님
- 결론적으로 차원을 축소하면 훈련 속도는 빨라지지만, 항상 더 낫거나 간단한 솔루션이 되는 것은 아님

##### PCA
- 훈련 세트에서 분산이 최대인 축을 찾고
- 첫 번째 축에 직교하고 남은 분산을 최대한 보전하는 두 번째 축을 찾음 ... n번째 축을 찾음
- 훈련 세트의 주성분은 특잇값 분해(SVD)라는 표준 행렬 분해 기술을 통해 구함

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(2)
X2D = pca.fit_transform(X)

##### 적절한 차원 수 선택하기

In [ ]:
# 기준을 정해서 선택 : 이를테면, 훈련 세트의 분산을 95%로 유지하는 데 필요한 최소한의 차원수를 선택
pca = PCA()
pca.fit(X_train)
cumsum = np.cumsum(pca.explained_variance_ratio)
d = np.argmax(cumsum >= 0.95) + 1
pca = PCA(n_components = d)

In [ ]:
# 하지만 이 보다는 유지하려는 분산 비율을 n_components에 넣는 게 훨씬 나음
pca = PCA(n_components=0.95)
X_reduced = pca.fit_transform(X_train)

##### 커널 PCA
- 5장의 커널트릭을 PCA에 적용시킨것

In [ ]:
from sklearn.decomposition import KernelPCA
rbf_pca = KernelPCA(n_components=2, kernel='rbf', gamma=0.04)
X_reduced = rbf_pca.fit_transform(X)

> 하이퍼파라미터 튜닝을 위해서는 GridSearchCV이후 모델에 학습시켜 최적값을 찾아내는 방식을 사용

##### 지역선형임베딩(LLE : Locally Linear Embedding)
- 비선형 차원 축소 기술
- 매니폴드 학습
- 간단히 말해 각 훈련 샘플이 가장 가까운 이웃에 얼마나 선형적으로 연관되어 있는지 측정
- 이후, 국부적인 관계가 가장 잘 보존되는 훈련세트의 저차원 표현을 찾는 방식임

In [ ]:
from sklearn.manifold import LocallyLinearEmbedding

lle = LocallyLinearEmbedding(n_components=2, n_neighbors=10)
X_reduced = lle.fit_transform(X)

##### 다른 차원 축소 기법
- 랜덤 투영(random projection) : 랜덤한 선형 투영을 통해 저차원화 = sklearn.random_projection
- lsomap : 각 샘플을 가까운 이웃과 연결하는 식으로 그래프를 만듦. 이후, 지오데식 거리를 유지하면서 차원 축소
- t-SNE : 비슷한 샘플은 가까이 그렇지 않은 샘플은 멀리 떨어지도록 하면서 차원을 축소, 주로 시각화에 많이 사용됨
- 선형 판별분석(LDA) : 분류알고리즘이지만, 훈련 과정에서 클래스 사이를 가장 잘 구분하는 축을 학습함 => SVM 분류기 같은 다른 분류 알고리즘을 적용하기 전에 차원을 축소하는 데 효과적임